In [14]:
# !pip3 install pypdf2

In [15]:
from PyPDF2 import PdfReader
import pandas as pd
import re
import pandas as pd

In [16]:
reader=PdfReader("Freakonomics 1.pdf","rb")
length=len(reader.pages)
pages=[]
for i in range(length):
    page=PdfReader._get_page(reader,page_number=i)
    pages.append(page)
    texts=[]
for i in range(length):
    text=pages[i].extract_text(0)
    texts.append(text)

In [17]:
contents=pd.DataFrame(columns=["pageNumber","content"])
data=[]
for num,val in enumerate(texts):
    data.append({"pageNumber":num,"content":val})
contents=pd.concat([contents,pd.DataFrame(data)],ignore_index=True)

In [18]:
import spacy
nlp=spacy.load("en_core_web_sm")
def formattedText(text):
    doc=nlp(text)
    page=[]
    for token in doc:
        token_index=token.i
        if token.text =="-":
            if token.i>0 and token.i<len(doc)-1:
                pageVal=doc[token.i-1].text+doc[token.i+1].text
            page.append(str(pageVal))
        elif token.text=="\n":
            pageVal=token.text.replace("\n","")
        elif token.text=="\n\n":
            pageVal=token.text.replace("\n\n","")
        # elif not token.is_punct:
        else:
            page.append(token.text)
    return " ".join(page)

In [19]:
# def join_letters(text):
#     doc=nlp(text)
#     page=[]
#     for token in doc[:-1]:
#         if len(doc[token.i+1])==1 and doc[token.i+1].text is not ["a","A","i","I"]: 
#             val=doc[token.i].text+doc[token.i+1].text
#             page.append(str(val))
#     return page

In [20]:
# contents.content=contents["content"].apply(join_letters)
contents.content=contents["content"].apply(formattedText)

In [21]:
# row=contents.iloc[5][1]
# print(row)

In [22]:
# !pip install transformers
# !pip install torch==1.9.0+cpu torchvision==0.10.0+cpu torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [24]:
from transformers import pipeline
modelName="sshleifer/distilbart-cnn-12-6"
RevisionName="a4f8f3e"
summarizer=pipeline("summarization",model=modelName,revision=RevisionName)
def add_summariser(text):
    return summarizer(text,min_length=150,max_length=200)

In [25]:
summarizedText=add_summariser(contents.iloc[5][1])
print(summarizedText)

[{'summary_text': ' The Bagel Man saw : mankind may be more honest than we think . How is the Ku Klux Klan like a group of real real estate agents? W h y experts of every kind are in the perfect position to exploit yo u ... Could sumo wrestling, the national sport of Japan be corrupt ? ... W h    a new car is suddenly worth so much less the moment it leaves the lo t ... A drug dealer, a high highschool school quarterback, and an editorial assistant have in commo n . Where Have All the Criminals Gone? 105 In which the facts of crime are sorted out from the ﬁctions . What Nicolae Ceaus ¸escu learned — the hard way — about abortio n'}]


In [26]:
normal_text=[text["summary_text"] for text in summarizedText]
normal_text=" ".join(normal_text)
print(normal_text)

 The Bagel Man saw : mankind may be more honest than we think . How is the Ku Klux Klan like a group of real real estate agents? W h y experts of every kind are in the perfect position to exploit yo u ... Could sumo wrestling, the national sport of Japan be corrupt ? ... W h    a new car is suddenly worth so much less the moment it leaves the lo t ... A drug dealer, a high highschool school quarterback, and an editorial assistant have in commo n . Where Have All the Criminals Gone? 105 In which the facts of crime are sorted out from the ﬁctions . What Nicolae Ceaus ¸escu learned — the hard way — about abortio n


In [ ]:
# contents["summarized"]=contents["content"].apply(add_summariser)